In [ ]:
import sqlite3
import hashlib
from datetime import datetime


########################################## INITIALISE LA BASE DE DONNEE "users.db" ##########################################

def init_db():
    conn = sqlite3.connect('user.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS users
                 (username TEXT PRIMARY KEY,
                  password TEXT,
                  registration_date TEXT)''')
    conn.commit()
    conn.close()


############################################### HASHAGE DU MDP ###############################################

def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()


############################################### ENREGISTREMENT DU CLIENT ###############################################

def register_user(username, password):
    """Retourne un message de succès ou d'erreur"""
    conn = sqlite3.connect('data.db')
    c = conn.cursor()
    try:
        c.execute("INSERT INTO users VALUES (?, ?, ?)",
                 (username, hash_password(password), datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
        conn.commit()
        return f"Compte {username} créé avec succès !"
    except sqlite3.IntegrityError:
        return "Erreur : Ce nom d'utilisateur existe déjà"
    finally:
        conn.close()


############################################### CONNECTION CLIENT ###############################################

def login_user(username, password):
    conn = sqlite3.connect('data.db')
    c = conn.cursor()
    c.execute("SELECT password FROM users WHERE username=?", (username,))
    result = c.fetchone()
    conn.close()
    
    if not result:
        return "Erreur : Utilisateur non trouvé"
    if result[0] != hash_password(password):
        return "Erreur : Mot de passe incorrect"
    return None  # Aucune erreur = connexion réussie


In [ ]:
import streamlit as st
import sqlite3
import bcrypt



####################################### CONNEXION BD POUR ENREGISTREMENT USER #######################################

# Connexion à la base de données
conn = sqlite3.connect("user.db", check_same_thread=False)
cursor = conn.cursor()
cursor.execute(
    """CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE NOT NULL,
        password BLOB NOT NULL
    )"""
)
conn.commit()

# Hachage et vérification du mot de passe
def hash_password(password):
    return bcrypt.hashpw(password.encode(), bcrypt.gensalt())

def check_password(password, hashed_password):
    return bcrypt.checkpw(password.encode(), hashed_password)

# Inscription
def register(username, password):
    cursor.execute("SELECT * FROM users WHERE username = ?", (username,))
    if cursor.fetchone():
        return "Nom d'utilisateur déjà pris"
    cursor.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, hash_password(password)))
    conn.commit()
    return "Compte créé avec succès !"

# Connexion
def login(username, password):
    cursor.execute("SELECT password FROM users WHERE username = ?", (username,))
    user = cursor.fetchone()
    if user and check_password(password, user[0]):
        st.session_state["user"] = username
        st.rerun()
    else:
        st.error("Identifiants incorrects")

# Déconnexion
def logout():
    st.session_state.pop("user", None)
    st.rerun()



####################################### STREAMLIT INTERFACE #######################################

# Chargement du fichier CSS
with open("src/assets/css/streamlit.css") as css:
    st.markdown(f"<style>{css.read()}</style>", unsafe_allow_html=True)

# CSS titre et sous-titre
st.markdown(f"""<div class="main-container"><h1>CONNEXION ET INSCRIPTION</h1></div>""", unsafe_allow_html=True)
st.markdown(f"""<div class="main-container"><h2>👤 Connexion</h2></div>""", unsafe_allow_html=True)

if "user" in st.session_state:
    st.success(f"Bienvenue, {st.session_state['user']} !")
    if st.button("Se déconnecter"):
        logout()
else:
    choice = st.radio("Connexion ou Inscription ?", ["Connexion", "Inscription"])
    username = st.text_input("Nom d'utilisateur")
    password = st.text_input("Mot de passe", type="password")

    if choice == "Inscription":
        confirm_password = st.text_input("Confirmez le mot de passe", type="password")
        if st.button("S'inscrire") and password == confirm_password:
            st.success(register(username, password))

    if choice == "Connexion":
        if st.button("Se connecter"):
            login(username, password) 

In [ ]:
ok pour ce code :
# models.py
import sqlite3
import bcrypt
from datetime import datetime
import streamlit as st

db_path = 'user.db'

# INITIALISE LA BASE DE DONNEE
def init_db():
    with sqlite3.connect(db_path) as conn:
        c = conn.cursor()
        c.execute('''
        CREATE TABLE IF NOT EXISTS users (
            username TEXT PRIMARY KEY,
            password TEXT,
            registration_date TEXT
        )
        ''')
        conn.commit()

# Hachage du mot de passe
def hash_password(password):
    return bcrypt.hashpw(password.encode(), bcrypt.gensalt()).decode('utf-8')

# Vérification du mot de passe
def check_password(password, hashed_password):
    return bcrypt.checkpw(password.encode(), hashed_password.encode('utf-8'))

# ENREGISTREMENT DU CLIENT
def register(username, password):
    with sqlite3.connect(db_path) as conn:
        c = conn.cursor()
        c.execute("SELECT username FROM users WHERE username = ?", (username,))
        if c.fetchone():
            return "❌ Nom d'utilisateur déjà pris"
        
        c.execute("INSERT INTO users (username, password, registration_date) VALUES (?, ?, ?)",
                  (username, hash_password(password), datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
        conn.commit()
        return f"✅ Compte '{username}' créé avec succès !"

# CONNECTION DU CLIENT
def login(username, password):
    with sqlite3.connect(db_path) as conn:
        c = conn.cursor()
        c.execute("SELECT password FROM users WHERE username=?", (username,))
        result = c.fetchone()
    
    if not result:
        return "Erreur : Utilisateur non trouvé"
    
    if not check_password(password, result[0]):
        return "Erreur : Mot de passe incorrect"
    
    return None  # Connexion réussie

# DECONNEXION
def logout():
    # Efface la session de l'utilisateur en cours
    st.session_state.pop('user', None)
    st.rerun()


avec : 
# controllers.py
import streamlit as st
from models import login, register, logout

def handle_login_register():
    if "user" in st.session_state:
        st.success(f"Bienvenue, {st.session_state.user} !")
        if st.button("Se déconnecter"):
            logout()  # Appel à la fonction de déconnexion
    else:
        username = st.text_input("Nom d'utilisateur")
        password = st.text_input("Mot de passe", type="password")

        if st.button("Se connecter"):
            error = login(username, password)
            if error:
                st.error(error)
            else:
                st.session_state.user = username
                st.rerun()

        if st.button("S'inscrire"):
            message = register(username, password)
            if "Erreur" in message:
                st.error(message)
            else:
                st.success(message)


et : 
# view.py
import streamlit as st
from controllers import handle_login_register

# Interface principale
with open("src/assets/css/streamlit.css") as css:
    st.markdown(f"<style>{css.read()}</style>", unsafe_allow_html=True)

st.markdown("""<div class="main-container"><h1>MISE À JOUR BASE DE DONNÉE</h1></div>""", unsafe_allow_html=True)

# Gérer l'interface utilisateur
handle_login_register()


Mais comment l'integrer au moein qui lui fait apparaitre une page si logué:
import sqlite3
import hashlib
import streamlit as st
from datetime import datetime
from src.models.users_db.main_db_users import *


########################################## INITIALISE LA BASE DE DONNEE "users.db" ##########################################


init_db()

########################################## DECONNEXION CLIENT ##########################################

def logout():
    st.session_state.pop('user', None) # Efface la session user de st.session_state
    st.rerun() # recharge la page


################################# CONNECTION CLIENT EN APPELANT "login_user()" ET GERE LA SESSION #################################

def login(username, password):
    error = login_user(username, password)
    if error:
        st.error(error)
    else:
        st.session_state.user = username
        st.rerun()


############################# ENREGISTREMENT DU CLIENT EN APPELANT "register_user()" ET AFFICHE RESULTAT #############################

def register(username, password):
    message = register_user(username, password)
    if "Erreur" in message:
        st.error(message)
    else:
        st.success(message)

# Interface principale
with open("src/assets/css/streamlit.css") as css:
    st.markdown(f"<style>{css.read()}</style>", unsafe_allow_html=True)

st.markdown("""<div class="main-container"><h1>MISE À JOUR BASE DE DONNÉE</h1></div>""", 
            unsafe_allow_html=True)


########################################## GERE  ##########################################
if "user" in st.session_state:
    st.success(f"Bienvenue, {st.session_state.user} !")
    if st.button("Se déconnecter"):
        logout()


    
# ===================================== PARTIE MISE À JOUR (visible seulement si connecté) ==================================== #
    
    # Chargement du fichier CSS
    with open("src/assets/css/streamlit.css") as css:
        st.markdown(f"<style>{css.read()}</style>", unsafe_allow_html=True)
    
    # CSS titre et sous-titre
    st.markdown(f"""<div class="main-container"><h1>MISE A JOUR BASE DE DONNEE</h1></div>""", unsafe_allow_html=True)
    st.markdown(f"""<div class="main-container"><h2>🔄 Mise à jour</h2></div>""", unsafe_allow_html=True)
    st.markdown(f"""<div class="main-container"><p>La mise à jour peut prendre entre 20 et 30 minutes</p></div>""", unsafe_allow_html=True)

    
    if st.button("Cliquez ici pour mettre à jour la base de données"):
        progress_bar = st.progress(0)
        
        try:
            # Étape 1/6
            progress_bar.progress(17)
            composition_indices.csv_indices(dossier_csv)
            st.write("✅ Étape 1 terminée - Scraping des tickers et composition des indices enregistrés")
            
            # Étape 2/6
            progress_bar.progress(34)
            infos_stocks.infos_stocks(dossier_csv)
            st.write("✅ Étape 2 terminée - Informations des entreprises enregistrées")
            
            # Étape 3/6
            progress_bar.progress(50)
            infos_indices.infos_indices(dossier_csv)
            st.write("✅ Étape 3 terminée - Informations des indices enregistrées")
            
            # Étape 4/6
            progress_bar.progress(67)
            hist_indices.recuperer_et_clean_indices(dossier_csv)
            st.write("✅ Étape 4 terminée - Historique des indices enregistré")
            
            # Étape 5/6
            progress_bar.progress(83)
            hist_stocks.recuperer_et_clean_stocks(dossier_csv)
            st.write("✅ Étape 5 terminée - Historique des entreprise enregistré")
            
            # Étape 6/6
            progress_bar.progress(100)
            sql_datas.main_creation_db(dossier_csv, db_path)
            st.write("✅ Étape 6 terminée - Base de donnée enregistrée")
            
            st.success("✅ ✅ Base de données mise à jour avec succès !")
            
        except Exception as e:
            st.error(f"❌ Erreur : {e}")
            progress_bar.progress(0)  # Réinitialise en cas d'erreur Configuration des utilisateurs (remplace par une base de 
                
# ===================================== FIN PARTIE MISE À JOUR (visible seulement si connecté) ==================================== #
    
else:
    st.markdown("""<div class="auth-container">""", unsafe_allow_html=True)
    choice = st.radio("Connexion ou Inscription ?", ["Connexion", "Inscription"],horizontal=True)
    
    username = st.text_input("Nom d'utilisateur")
    password = st.text_input("Mot de passe", type="password")
    
    if choice == "Inscription":
        confirm_password = st.text_input("Confirmez le mot de passe", type="password")
        if st.button("S'inscrire"):
            if password == confirm_password:
                register(username, password)
            else:
                st.error("Les mots de passe ne correspondent pas")
    
    if choice == "Connexion":
        if st.button("Se connecter"):
            login(username, password)
            
    
    st.markdown("""</div>""", unsafe_allow_html=True)

In [ ]:
# view.py
import streamlit as st
from models import login_user, register_user

# Interface principale
with open("src/assets/css/streamlit.css") as css:
    st.markdown(f"<style>{css.read()}</style>", unsafe_allow_html=True)

st.markdown("""<div class="main-container"><h1>MISE À JOUR BASE DE DONNÉE</h1></div>""", unsafe_allow_html=True)

# Vérifie si l'utilisateur est déjà connecté
if "user" in st.session_state:
    st.success(f"Bienvenue, {st.session_state.user} !")
    
    # Si l'utilisateur est connecté, afficher la section mise à jour de la base de données
    st.markdown(f"""<div class="main-container"><h2>🔄 Mise à jour</h2></div>""", unsafe_allow_html=True)
    st.markdown(f"""<div class="main-container"><p>La mise à jour peut prendre entre 20 et 30 minutes</p></div>""", unsafe_allow_html=True)
    
    if st.button("Cliquez ici pour mettre à jour la base de données"):
        progress_bar = st.progress(0)
        
        try:
            # Étapes de mise à jour de la base de données
            progress_bar.progress(17)
            composition_indices.csv_indices(dossier_csv)
            st.write("✅ Étape 1 terminée - Scraping des tickers et composition des indices enregistrés")
            
            progress_bar.progress(34)
            infos_stocks.infos_stocks(dossier_csv)
            st.write("✅ Étape 2 terminée - Informations des entreprises enregistrées")
            
            progress_bar.progress(50)
            infos_indices.infos_indices(dossier_csv)
            st.write("✅ Étape 3 terminée - Informations des indices enregistrées")
            
            progress_bar.progress(67)
            hist_indices.recuperer_et_clean_indices(dossier_csv)
            st.write("✅ Étape 4 terminée - Historique des indices enregistré")
            
            progress_bar.progress(83)
            hist_stocks.recuperer_et_clean_stocks(dossier_csv)
            st.write("✅ Étape 5 terminée - Historique des entreprises enregistré")
            
            progress_bar.progress(100)
            sql_datas.main_creation_db(dossier_csv, db_path)
            st.write("✅ Étape 6 terminée - Base de donnée enregistrée")
            
            st.success("✅ ✅ Base de données mise à jour avec succès !")
            
        except Exception as e:
            st.error(f"❌ Erreur : {e}")
            progress_bar.progress(0)  # Réinitialise en cas d'erreur

    if st.button("Se déconnecter"):
        # Déconnexion de l'utilisateur
        st.session_state.pop('user', None)
        st.rerun()  # Recharge la page pour supprimer la session de l'utilisateur

else:
    # Si l'utilisateur n'est pas connecté, afficher la page de connexion/inscription
    st.markdown("""<div class="auth-container">""", unsafe_allow_html=True)
    choice = st.radio("Connexion ou Inscription ?", ["Connexion", "Inscription"], horizontal=True)
    
    username = st.text_input("Nom d'utilisateur")
    password = st.text_input("Mot de passe", type="password")
    
    if choice == "Inscription":
        confirm_password = st.text_input("Confirmez le mot de passe", type="password")
        if st.button("S'inscrire"):
            if password == confirm_password:
                message = register_user(username, password)
                if "Erreur" in message:
                    st.error(message)
                else:
                    st.success(message)
            else:
                st.error("Les mots de passe ne correspondent pas")
    
    if choice == "Connexion":
        if st.button("Se connecter"):
            error = login_user(username, password)
            if error:
                st.error(error)
            else:
                st.session_state.user = username
                st.rerun()  # Recharge la page pour indiquer que l'utilisateur est connecté
            
    st.markdown("""</div>""", unsafe_allow_html=True)
